# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import os
import gmaps
# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [11]:
file = os.path.join("..", "WeatherPy","cities_weather_data.csv")
clean_city_data = pd.read_csv(file)

clean_city_data


,Unnamed: 0,City ID,City,Country,Latitude,Longitude,Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Date
0,0,3833367,Ushuaia,AR,-54.80,-68.30,46.40,63,40,18.34,1603042331
1,1,964432,Port Alfred,ZA,-33.59,26.89,63.00,79,50,9.95,1603042072
2,2,2075265,Busselton,AU,-33.65,115.33,61.00,49,3,9.48,1603042331
3,3,1067565,Beloha,MG,-25.17,45.05,66.06,77,0,18.28,1603042089
4,4,1254046,Tura,IN,25.52,90.22,74.25,89,0,2.28,1603042087
...,...,...,...,...,...,...,...,...,...,...,...
555,555,2525362,Carbonia,IT,39.17,8.52,64.99,77,40,10.29,1603042381
556,556,1283217,Khanbari,NP,27.37,87.20,68.22,62,25,1.81,1603042381
557,557,1501795,Krivosheino,RU,57.34,83.93,30.74,92,98,13.71,1603042381
558,558,5942913,Dryden,CA,49.78,-92.75,28.40,86,90,10.29,1603042382


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [12]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [13]:
# Use the Lat and Lng as locations and Humidity as the weight.
locations = clean_city_data[["Latitude", "Longitude"]].astype(float)
humidity = clean_city_data["Humidity (%)"].astype(float)

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [20]:
#My ideal weather condition is max temp between 70F to 90F, wind speed <5 mph, humidity 50 to 65%, and cloudiness <5%.

ideal_cities = clean_city_data.loc[(clean_city_data["Max Temp (F)"] > 70) & (clean_city_data["Max Temp (F)"] < 90) & (clean_city_data["Humidity (%)"] > 50) &  (clean_city_data["Humidity (%)"] < 65) & (clean_city_data["Wind Speed (mph)"] < 5) & (clean_city_data["Cloudiness (%)"] < 5), :]

ideal_cities


,Unnamed: 0,City ID,City,Country,Latitude,Longitude,Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Date
157,157,64435,Berbera,SO,10.44,45.01,85.78,55,0,4.74,1603042138
230,230,6201197,Muhafazat Lahij,YE,13.17,44.58,80.44,53,0,4.41,1603042351


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [23]:
hotel_df = ideal_cities

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
Berbera_coordinates = ("10.44, 45.01")
params = {
        "location": Berbera_coordinates,
        "keyword": "Hotel",
        "radius": 5000,
        "type": "lodging",
        "key": g_key
}
Berbera_data = requests.get(base_url, params=params).json()
#print(json.dumps(Berbera_data, indent=4, sort_keys=True))
hotel1 = Berbera_data["results"][0]["name"]
lon1 = Berbera_data["results"][0]["geometry"]["location"]["lng"]
lat1 = Berbera_data["results"][0]["geometry"]["location"]["lat"]
city1 = Berbera_data["results"][0]["plus_code"]["compound_code"]

ML_coordinates = ("13.17, 44.58")
params2 = {
        "location": Along_coordinates,
        "keyword": "Hotel",
        "radius": 5000,
        "type": "lodging",
        "key": g_key
}
ML_data = requests.get(base_url, params=params2).json()
#print(json.dumps(ML_data, indent=4, sort_keys=True))
hotel2 = ML_data["results"][0]["name"]
lon2 = ML_data["results"][0]["geometry"]["location"]["lng"]
lat2 = ML_data["results"][0]["geometry"]["location"]["lat"]
city2 = ML_data["results"][0]["plus_code"]["compound_code"]

hotel_df["Hotel Name"] = [hotel1, hotel2] 
hotel_df["Lng"] = [lon1, lon2]
hotel_df["Lat"] = [lat1, lat2]
hotel_df["City"] = ["Berbera", "Aalo"]
hotel_df["Country"] = ["Somalia", "Arunachal Pradesh"]

hotel_df

<ipython-input-23-1645e0f5304d>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = [hotel1, hotel2]
<ipython-input-23-1645e0f5304d>:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Lng"] = [lon1, lon2]
<ipython-input-23-1645e0f5304d>:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_gu

,Unnamed: 0,City ID,City,Country,Latitude,Longitude,Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Date,Hotel Name,Lng,Lat
157,157,64435,Berbera,Somalia,10.44,45.01,85.78,55,0,4.74,1603042138,Damal Hotel,45.008984,10.440097
230,230,6201197,Aalo,Arunachal Pradesh,13.17,44.58,80.44,53,0,4.41,1603042351,Hotel West,94.800352,28.168064


In [25]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

hotel_info

['\n<dl>\n<dt>Name</dt><dd>Damal Hotel</dd>\n<dt>City</dt><dd>Berbera</dd>\n<dt>Country</dt><dd>Somalia</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Hotel West</dd>\n<dt>City</dt><dd>Aalo</dd>\n<dt>Country</dt><dd>Arunachal Pradesh</dd>\n</dl>\n']

In [26]:
# Add marker layer ontop of heat map
# Display figure

#fig = gmaps.figure()
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
fig


Figure(layout=FigureLayout(height='420px'))